# <a name="hello-world"></a> A hello-world example of dflow

Define a Python OP

In [1]:
from pathlib import Path
from dflow.python import OP, Artifact

@OP.function
def duplicate(num: int, foo: Artifact(Path)) -> {"num": int, "bar": Artifact(Path)}:
    with open(foo, "r") as f:
        content = f.read()
    with open("bar.txt", "w") as f:
        f.write(content * 2)
    return {
        "num": num * 2,
        "bar": Path("bar.txt"),
    }

Upload the input file

In [2]:
from dflow import upload_artifact

with open("foo.txt", "w") as f:
    f.write("Hello")
art = upload_artifact("foo.txt")

Instantiate a step with the dispatcher executor and the uploaded artifact

In [3]:
from dflow.python import PythonOPTemplate
from dflow import Step

step1 = Step(
    name="step1",
    template=PythonOPTemplate(duplicate, image="python:3.8"),
    parameters={"num": 1},
    artifacts={"foo": art},
)

step2 = Step(
    name="step2",
    template=PythonOPTemplate(duplicate, image="python:3.8"),
    parameters={"num": step1.outputs.parameters["num"]},
    artifacts={"foo": step1.outputs.artifacts["bar"]},
)

Add step into a workflow

In [4]:
from dflow import Workflow

wf = Workflow(name="python-test")
wf.add(step1)
wf.add(step2)

Submit the workflow

In [5]:
wf.submit()

Workflow has been submitted (ID: python-test-2gcvt, UID: ddb20a1d-e8fb-4442-bbd3-602dc8fcfb1f)


{'metadata': {'name': 'python-test-2gcvt', 'generateName': 'python-test-', 'namespace': 'argo', 'uid': 'ddb20a1d-e8fb-4442-bbd3-602dc8fcfb1f', 'resourceVersion': '2206743', 'generation': 1, 'creationTimestamp': '2022-11-30T03:07:47Z', 'labels': {'workflows.argoproj.io/creator': 'system-serviceaccount-argo-argo-server'}, 'managedFields': [{'manager': 'argo', 'operation': 'Update', 'apiVersion': 'argoproj.io/v1alpha1', 'time': '2022-11-30T03:07:47Z', 'fieldsType': 'FieldsV1', 'fieldsV1': {'f:metadata': {'f:generateName': {}, 'f:labels': {'.': {}, 'f:workflows.argoproj.io/creator': {}}}, 'f:spec': {}, 'f:status': {}}}]}, 'spec': {'templates': [{'name': 'python-test-steps', 'inputs': {}, 'outputs': {}, 'metadata': {}, 'steps': [[{'name': 'step1', 'template': 'duplicate-cpxog', 'arguments': {'parameters': [{'name': 'num', 'value': '1', 'description': '{"type": "<class \'int\'>"}'}], 'artifacts': [{'name': 'foo', 'path': '/tmp/inputs/artifacts/foo', 's3': {'key': 'upload/ba6a4ae7-5735-4bf6-a

# <a name="Argo-UI"></a> Argo UI

After finishing the previous steps, we can access the workflow we just ran on the UI (https://127.0.0.1:2746)